# Segmenting and Clustering Neighborhoods in Toronto

Neighborhoods data is processed to Pandas dataframe from notebook :
https://github.com/EdwinLife/Coursera_Capstone/blob/master/Explore%20Neighborhoods%20in%20Toronto%20-%20Web%20Scraping.ipynb

Here latitude and longitude of Toronto's neighborhoods is obtained from csv file referred in assignment. 

### Code in first Notebook scraping wikipedia table to Pandas dataframe.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

#scrape table from wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
soup = BeautifulSoup(requests.get(url).content, 'lxml')
table = soup.find_all('table')[0] 
df = pd.DataFrame()

header = []
for th in table.find_all("th"):
    # remove new line and spaces from left and right        
    header.append(th.text.replace('\n', ' ').strip())

for tr in table.tbody.find_all("tr"): # find all <tr> from table's tbody
    t_row = {}
    add_row = True
    for td, th in zip(tr.find_all("td"), header): 
        t_row[th] = td.text.replace('\n', '').strip()
        if th == 'Borough' and (t_row[th] == 'Not assigned' or t_row[th] == ''):
            add_row = False
        
    if add_row:
        df = df.append(t_row, ignore_index=True)

df = df[header]
df = df.groupby(['Postal code','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df['Neighborhood'] = df['Neighborhood'].str.replace('/',',')
df.rename(columns={'Postal code':'Postal Code'}, inplace='True')
df.loc[df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df['Borough']
df.shape

(103, 3)

### Code in second Notebook : Get geographical coordinates of each postal code

Geocoder Python package instead: https://geocoder.readthedocs.io/index.html seems to be unreliable as mentioned in Assignment instruction, hence the csv file mentioned is used - that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [2]:
import numpy as np
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.head()

df_join = pd.merge(df, df_geo, on='Postal Code', how='left')
df_join.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848


### Import Packages

In [3]:
# Get Toronto latitude and longitude
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



## 1. Download and Explore Dataset

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.
Just make sure:
1. to add enough Markdown cells to explain what you decided to do and to report any observations you make.
2. to generate maps to visualize your neighborhoods and how they cluster together.

Only boroughs that contain the word Toronto to be explored

In [4]:
df_toronto = df_join[df_join.Borough.str.contains('Toronto',case=False)]
df_toronto.reset_index()

,index,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,41,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188
2,42,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923
4,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,47,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,48,M4T,Central Toronto,"Moore Park , Summerhill East",43.689574,-79.383160
9,49,M4V,Central Toronto,"Summerhill West , Rathnelly , South Hill , For...",43.686412,-79.400049


Toronto's latitude and longitude

In [5]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [6]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Explore further Downtown Toronto only

In [7]:
df_downtown = df_toronto[df_toronto['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
df_downtown.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town , Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [8]:
address = 'Downtown Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


In [9]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, label in zip(df_downtown['Latitude'], df_downtown['Longitude'], df_downtown['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

#### Explore Downtown with Foursquare

In [10]:
# The code was removed by Watson Studio for sharing.

#### Explore Downtown's Rosedale neighborhood

In [11]:
# Get neighborhood name and geocode
print('Exploring neighborhood : ', df_downtown.loc[0, 'Neighborhood'])
neighborhood_latitude = df_downtown.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_downtown.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_downtown.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Exploring neighborhood :  Rosedale
Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


Get Top 50 venues in Rosedale neighborhood

In [14]:
LIMIT = 50
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

results = requests.get(url).json()

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
# clean the json and structure it into a pandas dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Rosedale Park,Playground,43.682328,-79.378934
1,Whitney Park,Park,43.682036,-79.373788
2,Alex Murray Parkette,Park,43.678300,-79.382773
3,Milkman's Lane,Trail,43.676352,-79.373842


In [18]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


## Explore Neighborhoods in Downtown

#### Function for all neigborhoods

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
downtown_venues = getNearbyVenues(names= df_downtown['Neighborhood'],
                                   latitudes=df_downtown['Latitude'],
                                   longitudes=df_downtown['Longitude']
                                  )

Rosedale
St. James Town , Cabbagetown
Church and Wellesley
Regent Park , Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond , Adelaide , King
Harbourfront East , Union Station , Toronto Islands
Toronto Dominion Centre , Design Exchange
Commerce Court , Victoria Hotel
University of Toronto , Harbord
Kensington Market , Chinatown , Grange Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Stn A PO Boxes
First Canadian Place , Underground city
Christie
Queen's Park , Ontario Provincial Government


Check Downtown venues size

In [21]:
print(downtown_venues.shape)
downtown_venues.head()

(792, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"St. James Town , Cabbagetown",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


Check Downtown venues size per neighborhood and unique categories

In [22]:
print(downtown_venues.groupby('Neighborhood').count())
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

                                                    Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            50   
CN Tower , King and Spadina , Railway Lands , H...                     16   
Central Bay Street                                                     50   
Christie                                                               18   
Church and Wellesley                                                   50   
Commerce Court , Victoria Hotel                                        50   
First Canadian Place , Underground city                                50   
Garden District, Ryerson                                               50   
Harbourfront East , Union Station , Toronto Isl...                     50   
Kensington Market , Chinatown , Grange Park                            50   
Queen's Park , Ontario Provincial Government                           32   

## 3. Analyze Each Neighborhood in Downtown

In [23]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
# Examine the new dataframe size.
downtown_onehot.shape

(792, 175)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [25]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.000000,...,0.00,0.020,0.000000,0.00,0.00,0.00,0.02,0.000000,0.00,0.00
1,"CN Tower , King and Spadina , Railway Lands , ...",0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.00,0.000000,...,0.00,0.000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00
2,Central Bay Street,0.020000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.02,0.000000,...,0.02,0.000,0.000000,0.00,0.00,0.00,0.02,0.000000,0.00,0.02
3,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.000000,...,0.00,0.000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00
4,Church and Wellesley,0.040000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.000000,...,0.00,0.020,0.020000,0.02,0.00,0.00,0.00,0.000000,0.00,0.00
5,"Commerce Court , Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.04,0.000000,...,0.02,0.000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.02
6,"First Canadian Place , Underground city",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.02,0.000000,...,0.04,0.000,0.000000,0.00,0.00,0.00,0.02,0.000000,0.00,0.02
7,"Garden District, Ryerson",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.02,0.000000,...,0.02,0.020,0.040000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00
8,"Harbourfront East , Union Station , Toronto Is...",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.000000,...,0.00,0.000,0.000000,0.00,0.00,0.02,0.00,0.000000,0.00,0.00
9,"Kensington Market , Chinatown , Grange Park",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.000000,...,0.00,0.000,0.000000,0.00,0.00,0.00,0.06,0.000000,0.04,0.02


In [26]:
# new size after grouped
downtown_grouped.shape

(19, 175)

#### Each neighborhood along with the top 5 most common venues

In [27]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.06
1            Beer Bar  0.04
2  Seafood Restaurant  0.04
3              Bakery  0.04
4      Farmers Market  0.04


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3          Boutique  0.06
4               Bar  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1  Italian Restaurant  0.06
2        Burger Joint  0.04
3                Café  0.04
4     Bubble Tea Shop  0.04


----Christie----
           venue  freq
0  Grocery Store  0.22
1           Café  0.17
2           Park  0.11
3     Baby Store  0.06
4      Nightclub  0.06


----Church and Wellesley----
          venue  freq
0   Coffee Shop  0.06
1       Gay Bar  0.06
2   Yoga Studio  0.04
3  Burger Joint  0.04
4   Men's Store  0.04


----Commerce C

#### Put into dataframe

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Beer Bar,Seafood Restaurant,Cocktail Bar,Bakery,Restaurant,Cheese Shop,Farmers Market,Café,Hotel
1,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Lounge,Airport Terminal,Plane,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Boutique,Bar
2,Central Bay Street,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Sandwich Place,Bubble Tea Shop,Burger Joint,Café,Spa,Wine Bar,Modern European Restaurant
3,Christie,Grocery Store,Café,Park,Diner,Athletics & Sports,Coffee Shop,Gas Station,Candy Store,Restaurant,Italian Restaurant
4,Church and Wellesley,Coffee Shop,Gay Bar,Yoga Studio,Men's Store,Restaurant,Burger Joint,Gastropub,Park,Beer Bar,Ramen Restaurant


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [30]:
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 4, 2, 0, 0, 0, 0, 0, 0], dtype=int32)

In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = df_downtown

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Trail,Playground,Wine Bar,Colombian Restaurant,Deli / Bodega,Dance Studio,Creperie,Cosmetics Shop,Convenience Store
1,M4X,Downtown Toronto,"St. James Town , Cabbagetown",43.667967,-79.367675,0,Coffee Shop,Café,Italian Restaurant,Pub,Pizza Place,Bakery,Restaurant,Convenience Store,Indian Restaurant,Gastropub
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Gay Bar,Yoga Studio,Men's Store,Restaurant,Burger Joint,Gastropub,Park,Beer Bar,Ramen Restaurant
3,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,4,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Theater,Café,Mexican Restaurant,Yoga Studio,Cosmetics Shop
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Café,Clothing Store,Cosmetics Shop,Restaurant,Ramen Restaurant,Theater,Italian Restaurant,Bookstore,Hotel


### Visualize the resulting clusters

In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

#### Cluster 1

In [33]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,0,Coffee Shop,Café,Italian Restaurant,Pub,Pizza Place,Bakery,Restaurant,Convenience Store,Indian Restaurant,Gastropub
2,Downtown Toronto,0,Coffee Shop,Gay Bar,Yoga Studio,Men's Store,Restaurant,Burger Joint,Gastropub,Park,Beer Bar,Ramen Restaurant
4,Downtown Toronto,0,Coffee Shop,Café,Clothing Store,Cosmetics Shop,Restaurant,Ramen Restaurant,Theater,Italian Restaurant,Bookstore,Hotel
5,Downtown Toronto,0,Café,Coffee Shop,Beer Bar,Park,Restaurant,Hotel,Japanese Restaurant,Gastropub,Farmers Market,Cosmetics Shop
6,Downtown Toronto,0,Coffee Shop,Beer Bar,Seafood Restaurant,Cocktail Bar,Bakery,Restaurant,Cheese Shop,Farmers Market,Café,Hotel
8,Downtown Toronto,0,Coffee Shop,Café,Steakhouse,Bar,Restaurant,Asian Restaurant,American Restaurant,Pizza Place,Deli / Bodega,Salad Place
9,Downtown Toronto,0,Coffee Shop,Aquarium,Bar,Park,Café,Plaza,Hotel,Basketball Stadium,Salad Place,Restaurant
10,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Gastropub,Seafood Restaurant,Japanese Restaurant,Hotel,Beer Bar,Concert Hall,Ice Cream Shop
11,Downtown Toronto,0,Coffee Shop,Café,Gastropub,Hotel,Restaurant,Seafood Restaurant,Japanese Restaurant,Bakery,Gym,Beer Bar
12,Downtown Toronto,0,Café,Yoga Studio,Bar,Italian Restaurant,Japanese Restaurant,Restaurant,Bookstore,Bakery,Sandwich Place,Beer Bar


#### Cluster 2

In [34]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Park,Trail,Playground,Wine Bar,Colombian Restaurant,Deli / Bodega,Dance Studio,Creperie,Cosmetics Shop,Convenience Store


#### Cluster 3

In [35]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,2,Grocery Store,Café,Park,Diner,Athletics & Sports,Coffee Shop,Gas Station,Candy Store,Restaurant,Italian Restaurant


#### Cluster 4

In [36]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,3,Airport Service,Airport Lounge,Airport Terminal,Plane,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Boutique,Bar


#### Cluster 5

In [37]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Toronto,4,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Theater,Café,Mexican Restaurant,Yoga Studio,Cosmetics Shop
7,Downtown Toronto,4,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Sandwich Place,Bubble Tea Shop,Burger Joint,Café,Spa,Wine Bar,Modern European Restaurant
18,Downtown Toronto,4,Coffee Shop,Diner,Hobby Shop,Bar,Distribution Center,Discount Store,Sandwich Place,Boutique,Juice Bar,Creperie
